In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
#url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports/current/countrybycommodityimports11032021144801.zip"


url = "https://www.ons.gov.uk/file?uri=/economy/nationalaccounts/balanceofpayments/datasets/uktradecountrybycommodityimports/current/countrybycommodityimports.zip"

In [3]:
r = requests.get(url) 

brexit_FEB21 = zf.ZipFile(io.BytesIO(r.content))

brexit_FEB21.extractall(".\\data")

filename = brexit_FEB21.namelist()

In [4]:
df = pd.read_excel(".\\data" + "\\" + filename[0], sheet_name= "Country by Commodity Import")

In [5]:
df.head()

,COMMODITY,COUNTRY,DIRECTION,1997JAN,1997FEB,1997MAR,1997APR,1997MAY,1997JUN,1997JUL,...,2020MAY,2020JUN,2020JUL,2020AUG,2020SEP,2020OCT,2020NOV,2020DEC,2021JAN,2021FEB
0,0 Food & live animals,AD Andorra,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0 Food & live animals,AE United Arab Emirates,IM Imports,0.27,0.07,2.51,22.37,5.06,0.23,0.61,...,8.70,1.24,1.48,1.63,0.84,1.02,1.46,1.18,2.19,1.21
2,0 Food & live animals,AF Afghanistan,IM Imports,0.06,0.07,0.00,0.23,0.05,0.05,0.00,...,0.00,0.00,0.00,0.02,0.11,0.04,0.12,0.10,0.11,0.24
3,0 Food & live animals,AG Antigua & Barbuda,IM Imports,0.02,0.12,0.00,0.00,0.04,0.01,0.00,...,0.01,0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0 Food & live animals,AI Anguilla,IM Imports,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [6]:
df = df.melt(id_vars = ["COMMODITY", "COUNTRY", "DIRECTION"])

df.drop("DIRECTION", axis = 1, inplace = True)

df.variable= pd.to_datetime(df.variable, format="%Y%b")

df.rename({"variable":"time", "value": "imports"},inplace = True, axis = 1)

df.COUNTRY = df.COUNTRY.str[3:]

df.imports = df.imports*1000000

In [7]:
grp = df[df.COMMODITY == 'T Total'].groupby("COUNTRY")

test = grp.agg({"imports": "sum"})

In [8]:
topcntry = test.sort_values(by = "imports", ascending = False).iloc[0:23].index.to_list()

In [9]:
df = df[df.COUNTRY.isin(topcntry)]

In [10]:
df.COMMODITY.replace({"T Total": "Total"}, inplace = True)
df.COUNTRY.replace({"United States inc Puerto Rico": "United States"}, inplace = True)
df.COUNTRY.replace({"Extra EU 28 (Rest of World)": "Excl. EU 28 (Rest of World)"}, inplace = True)

In [11]:
df.tail()

,COMMODITY,COUNTRY,time,imports
8591191,Total,Russia,2021-02-01,1.213000e+09
8591197,Total,Sweden,2021-02-01,4.770000e+08
8591224,Total,Turkey,2021-02-01,6.620000e+08
8591232,Total,United States,2021-02-01,3.708000e+09
8591242,Total,Whole world,2021-02-01,3.867700e+10


In [12]:
df.set_index(["COUNTRY", "COMMODITY", "time"], inplace = True)

In [13]:
df.head()

,,,imports
COUNTRY,COMMODITY,time,
Total EU(28),0 Food & live animals,1997-01-01,673940000.0
Belgium,0 Food & live animals,1997-01-01,44340000.0
Canada,0 Food & live animals,1997-01-01,14320000.0
Switzerland,0 Food & live animals,1997-01-01,4110000.0
China,0 Food & live animals,1997-01-01,6530000.0


In [14]:
out_file = ".\\data"+ "\\UK-imports.parquet"

pq.write_table(pa.Table.from_pandas(df.sort_index()), out_file)

In [15]:
file = ".\\data"+ "\\UK-imports.parquet"

foo = pq.read_table(file).to_pandas()

In [17]:
foo.tail()

imports
COUNTRY     COMMODITY time                    
Whole world Total     2020-10-01  4.246900e+10
                      2020-11-01  4.391400e+10
                      2020-12-01  4.074500e+10
                      2021-01-01  3.376800e+10
                      2021-02-01  3.867700e+10